In [7]:
from pulp import *
import pandas as pd

In [8]:
DniTygodnia = ["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek", "Sobota", "Niedziela"]
Pracownicy = ["Igor", "Marcin", "Franek", "Piotr", "Ania", "Zenon"]

## zwraca sumę pensji wszystkich pracowników
def PensjeLacznie():
    suma = 0
    for imie in Pracownicy:
            for dzien in DniTygodnia:
                suma += GodzinyPracy[imie][dzien] * Stawki[imie]
    return suma

## zwraca sumę godzin otwarcia w ciągu tygodnia
def GodzinyOtwarciaLacznie():
    suma = 0
    for dzien in DniTygodnia:
        suma += OpeningHours[dzien]
    return suma

## zwraca sumę wypracowanych godzin przez wszystkich pracownikow
def WypracowaneGodzinyLacznie():
    suma = 0
    for dzien in DniTygodnia:
        suma += SumaGodzinDzien(dzien)
    return suma

## zwraca sumę wypracowanych godzin przez pracownika
def SumaGodzinPracownik(imie):
    suma = 0
    for dzien in DniTygodnia:
        suma += GodzinyPracy[imie][dzien]
    return suma

## zwraca sumę wypracowanych godzin danego dnia (przez wszystkich pracowników)
def SumaGodzinDzien(dzien):
    suma = 0
    for imie in Pracownicy:
        suma += GodzinyPracy[imie][dzien]
    return suma

prob = LpProblem("Najnizszy koszt pracownikow", LpMinimize)

GodzinyPracy = {}
for imie in Pracownicy:
    GodzinyPracy[imie] = {}

## slownik dla stawek
Stawki = {"Igor": 170, "Marcin": 60, "Franek": 80, "Piotr": 200, "Ania": 90, "Zenon": 18}

## slownik dla dostepnosci
Dostepnosc = {"Igor": 4, "Marcin": 4, "Franek": 9, "Piotr": 8, "Ania": 10, "Zenon": 5}
Availability = Dostepnosc.copy()

## slownik dla godzin otwarcia
GodzinyOtwarcia = {"Poniedzialek": 8, "Wtorek": 4, "Sroda": 4, "Czwartek": 4, "Piatek": 8, "Sobota": 2, "Niedziela": 0}
OpeningHours = GodzinyOtwarcia.copy()

for imie in Pracownicy:
    
    ## zadeklarowanie zmiennych dostepnosci dla każdego pracownika
    Availability[imie] = LpVariable("Godziny dostepnosci " + imie, 0)
    
    ## przypisanie zmiennym odpowiednich wartości
    prob += Availability[imie] == Dostepnosc[imie], imie + " Dostepne godziny "
    
    for dzien in DniTygodnia:
        
        ## zadeklarowanie zmiennych godzin pracujacych dla kazdego pracownika
        GodzinyPracy[imie][dzien] = LpVariable("Godziny pracujace " + imie + " " + dzien, 0)
        
        ## warunek, po to zeby polecenia wykonaly sie tylko raz na koniec (żeby nie robić oddzielnej pętli)
        if imie == Pracownicy[len(Pracownicy) - 1]:
            
            ## zadeklarowanie zmiennych godzin otwarcia dla kazdego dnia 
            OpeningHours[dzien] = LpVariable("Godziny otwarcia w " + dzien, 0)
            
            ## przypisanie zmiennym odpowiednich wartości
            prob += OpeningHours[dzien] == GodzinyOtwarcia[dzien], dzien + " Godziny otwarcia"
            
            ## zadeklarowanie, że suma godzin przepracowanych przez pracownika danego dnia ma być równa godzinom otwarcia
            prob += SumaGodzinDzien(dzien) == OpeningHours[dzien] #7
            
## dodanie ogarniczen -> Suma godzin pracujących pracownika musi być mniejsza lub równa jego dostępności
for imie in Pracownicy:
    prob += SumaGodzinPracownik(imie) <= Availability[imie]

## zadeklarowanie, że suma wypracowanych godzin przez wszystkich pracowników musi być równa łącznej ilości godzin, w których sklep był otwarty
prob += GodzinyOtwarciaLacznie() == WypracowaneGodzinyLacznie(), "Godziny pracy"

## równanie główne, obliczające łączny koszt pracowników
prob += PensjeLacznie()

In [9]:
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)
    
print("-"*20)   
print("Najnizszy koszt pracowników {} zł  ".format( value(prob.objective)))
print("-"*20)

Status: Optimal
Godziny_dostepnosci_Ania = 10.0
Godziny_dostepnosci_Franek = 9.0
Godziny_dostepnosci_Igor = 4.0
Godziny_dostepnosci_Marcin = 4.0
Godziny_dostepnosci_Piotr = 8.0
Godziny_dostepnosci_Zenon = 5.0
Godziny_otwarcia_w_Czwartek = 4.0
Godziny_otwarcia_w_Piatek = 8.0
Godziny_otwarcia_w_Poniedzialek = 8.0
Godziny_otwarcia_w_Sobota = 2.0
Godziny_otwarcia_w_Sroda = 4.0
Godziny_otwarcia_w_Wtorek = 4.0
Godziny_pracujace_Ania_Piatek = 8.0
Godziny_pracujace_Ania_Sobota = 1.0
Godziny_pracujace_Ania_Sroda = 1.0
Godziny_pracujace_Franek_Czwartek = 4.0
Godziny_pracujace_Franek_Sobota = 1.0
Godziny_pracujace_Franek_Wtorek = 4.0
Godziny_pracujace_Igor_Poniedzialek = 2.0
Godziny_pracujace_Marcin_Poniedzialek = 4.0
Godziny_pracujace_Zenon_Poniedzialek = 2.0
Godziny_pracujace_Zenon_Sroda = 3.0
--------------------
Najnizszy koszt pracowników 2290.0 zł  
--------------------
